# Project - Display Cancelled flights for US Airlines in US for 2015

---
**Data Source:** US Department of Transportation as available from kaggle.com (https://www.kaggle.com/usdot/flight-delays)
> **Context:**
The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics tracks the on-time performance of domestic flights operated by large air carriers. Summary information on the number of on-time, delayed, canceled, and diverted flights is published in DOT's monthly Air Travel Consumer Report and in this dataset of 2015 flight delays and cancellations.

> **Acknowledgement:**
The flight delay and cancellation data was collected and published by the DOT's Bureau of Transportation Statistics.

---

## Step 1: Import the raw data into Pandas DataFrame

In [27]:
# Import dependencies and Libraries
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
import json

In [28]:
# Import API key
from api_keys import gkey

In [2]:
# Import the Cancellation Code data file as DataFrame
csv_file = "data/cancellation_code.csv"
cancellation_code_df = pd.read_csv(csv_file)
cancellation_code_df.head()

,CANCELLATION_REASON_CODE,CANCELLATION_REASON
0,A,Airline/Carrier
1,B,Weather
2,C,National Air System
3,D,Security


In [3]:
# Import the raw US Airlines data file as DataFrame
csv_file = "data/airlines.csv"
airlines_df = pd.read_csv(csv_file)
airlines_df.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [4]:
# Import the raw US Airports data file as DataFrame
csv_file = "data/airports.csv"
airports_df = pd.read_csv(csv_file)
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [5]:
# Import the raw US flights Arrival/Departure data for US Airlines data file as DataFrame
csv_file = "data/flights_clean.csv"
flights_df = pd.read_csv(csv_file)
flights_df

,level_0,index,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,32,32,2015,1,1,4,AS,136,N431AS,ANC,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
1,42,42,2015,1,1,4,AA,2459,N3BDAA,PHX,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
2,68,68,2015,1,1,4,OO,5254,N746SK,MAF,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
3,82,82,2015,1,1,4,MQ,2859,N660MQ,SGF,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
4,90,90,2015,1,1,4,OO,5460,N583SW,RDD,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87425,5818090,5818090,2015,12,31,4,UA,1789,NaN,IAH,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
87426,5818157,5818157,2015,12,31,4,UA,222,NaN,SFO,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
87427,5818318,5818318,2015,12,31,4,AA,2245,N880AA,MIA,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
87428,5818777,5818777,2015,12,31,4,NK,416,N522NK,FLL,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN


## Step 2: Cleanse and Extract the data
This step will include all data cleansing, recreating the the DataFrame with required data and renaming of columns.

In [6]:
flights_df.count()

level_0                87430
index                  87430
YEAR                   87430
MONTH                  87430
DAY                    87430
DAY_OF_WEEK            87430
AIRLINE                87430
FLIGHT_NUMBER          87430
TAIL_NUMBER            73063
ORIGIN_AIRPORT         87430
DESTINATION_AIRPORT    87430
SCHEDULED_DEPARTURE    87430
DEPARTURE_TIME          3616
DEPARTURE_DELAY         3616
TAXI_OUT                 818
WHEELS_OFF               818
SCHEDULED_TIME         87425
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE               87430
WHEELS_ON                  0
TAXI_IN                    0
SCHEDULED_ARRIVAL      87430
ARRIVAL_TIME               0
ARRIVAL_DELAY              0
DIVERTED               87430
CANCELLED              87430
CANCELLATION_REASON    87430
AIR_SYSTEM_DELAY           0
SECURITY_DELAY             0
AIRLINE_DELAY              0
LATE_AIRCRAFT_DELAY        0
WEATHER_DELAY              0
dtype: int64

In [7]:
# Prepping to extract the data related to Flight cancellations only
flights_df.dropna(subset=['CANCELLATION_REASON'], how='all', inplace=True)
flights_df.count()

level_0                87430
index                  87430
YEAR                   87430
MONTH                  87430
DAY                    87430
DAY_OF_WEEK            87430
AIRLINE                87430
FLIGHT_NUMBER          87430
TAIL_NUMBER            73063
ORIGIN_AIRPORT         87430
DESTINATION_AIRPORT    87430
SCHEDULED_DEPARTURE    87430
DEPARTURE_TIME          3616
DEPARTURE_DELAY         3616
TAXI_OUT                 818
WHEELS_OFF               818
SCHEDULED_TIME         87425
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE               87430
WHEELS_ON                  0
TAXI_IN                    0
SCHEDULED_ARRIVAL      87430
ARRIVAL_TIME               0
ARRIVAL_DELAY              0
DIVERTED               87430
CANCELLED              87430
CANCELLATION_REASON    87430
AIR_SYSTEM_DELAY           0
SECURITY_DELAY             0
AIRLINE_DELAY              0
LATE_AIRCRAFT_DELAY        0
WEATHER_DELAY              0
dtype: int64

In [8]:
# further clean the DatFrame the 'ORIGIN_AIRPORT_CODE and 'DESTINATION_AIRPORT_CODE' 
# columns have numeric values that are not codes and not in the Airports table.

del flights_df["level_0"]
del flights_df["index"]

boolean = []
# Check if the value is Alpha
for airport in flights_df.ORIGIN_AIRPORT:
    if str(airport).isalpha():
        boolean.append(True)
    else:
        boolean.append(False)

#len(boolean_alpha)
#boolean_alpha[-1]

# Convert the boolena list to a Panda Series
is_alpha = pd.Series(boolean)

# Filter out the numeric value and retain the rows with ORIGIN_AIRPORT_CODE that are Alpha.
flights_df = flights_df[is_alpha]

In [13]:
flights_df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,200,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,OO,5254,N746SK,MAF,IAH,510,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,MQ,2859,N660MQ,SGF,DFW,525,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,OO,5460,N583SW,RDD,SFO,530,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87425,2015,12,31,4,UA,1789,NaN,IAH,TPA,1955,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
87426,2015,12,31,4,UA,222,NaN,SFO,LAX,2000,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
87427,2015,12,31,4,AA,2245,N880AA,MIA,SAN,2019,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
87428,2015,12,31,4,NK,416,N522NK,FLL,IAG,2155,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN


In [14]:
# Extract only required columns and create a new dataframe
flights_selected_df = flights_df[["YEAR", "MONTH", "DAY", "DAY_OF_WEEK","AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "CANCELLATION_REASON"]]

# Rename some columns
flights_selected_df = flights_selected_df.rename(columns={"AIRLINE":"AIRLINE_CODE", "ORIGIN_AIRPORT":"ORIGIN_AIRPORT_CODE","DESTINATION_AIRPORT":"DESTINATION_AIRPORT_CODE","CANCELLATION_REASON":"CANCELLATION_REASON_CODE"})
flights_selected_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_CODE,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,CANCELLATION_REASON_CODE
0,2015,1,1,4,AS,ANC,SEA,A
1,2015,1,1,4,AA,PHX,DFW,B
2,2015,1,1,4,OO,MAF,IAH,B
3,2015,1,1,4,MQ,SGF,DFW,B
4,2015,1,1,4,OO,RDD,SFO,A


In [15]:
# Add a 'DATE" column by combining the 'YEAR, 'MONTH and 'DAY' columns 
flights_selected_df["DATE"] = pd.to_datetime(flights_selected_df["YEAR"].astype("str") + '-'+ flights_selected_df["MONTH"].astype("str")+'-'+flights_selected_df["DAY"].astype("str"))
# Create a new Dataframe  taht includes 'DATE' and ignores the constituents columns
cancelled_flights_df = flights_selected_df[["DATE", "AIRLINE_CODE", "ORIGIN_AIRPORT_CODE", "DESTINATION_AIRPORT_CODE", "CANCELLATION_REASON_CODE"]]
cancelled_flights_df.head()

,DATE,AIRLINE_CODE,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,CANCELLATION_REASON_CODE
0,2015-01-01,AS,ANC,SEA,A
1,2015-01-01,AA,PHX,DFW,B
2,2015-01-01,OO,MAF,IAH,B
3,2015-01-01,MQ,SGF,DFW,B
4,2015-01-01,OO,RDD,SFO,A


In [16]:
cancelled_flights_df.reset_index(inplace=True)
del cancelled_flights_df['index']
cancelled_flights_df.reset_index(inplace=True)

# Rename the Index column to ID as this will be used as PK in the table
cancelled_flights_df = cancelled_flights_df.rename(columns={'index': 'ID'})

# display the header records, this Datframe is ready to be loaded into the database
cancelled_flights_df.head()

,ID,DATE,AIRLINE_CODE,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,CANCELLATION_REASON_CODE
0,0,2015-01-01,AS,ANC,SEA,A
1,1,2015-01-01,AA,PHX,DFW,B
2,2,2015-01-01,OO,MAF,IAH,B
3,3,2015-01-01,MQ,SGF,DFW,B
4,4,2015-01-01,OO,RDD,SFO,A


## Step 3: Load the cleansed data into the Postgres database
This step will load the 4 tables in the database 'flight_db' that will be created in Postgres based on the ER diagram

In [17]:
# Connect to Local Postgres Database server
#rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
rds_connection_string = "postgres:postgres@localhost:5432/flight_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
# Check for 4 Tables as they should exist and be empty
engine.table_names()
session = sessionmaker(bind=engine, autocommit=True, autoflush=False)()

In [ ]:
# Use Pandas to load the DataFrames into database (Converted from raw csv files and cleansed)
# Load the 'cancellation_code' table
cancellation_code_df.to_sql(name='cancellation_code', con=engine, if_exists='append', index=False)
print(f"Loaded {session.execute('select count(*) from cancellation_code;').scalar():,} records to cancellation_code table")

# Load the 'airlines' table
airlines_df.to_sql(name='airlines', con=engine, if_exists='append', index=False)
print(f"Loaded {session.execute('select count(*) from airlines;').scalar():,} records to airlines table")

# Load the 'airports' table
airports_df.to_sql(name='airports', con=engine, if_exists='append', index=False)
print(f"Loaded {session.execute('select count(*) from airports;').scalar():,} records to airport table")

# Load the 'cancelled_flights' table (To be loaded last as this table has Fk dependency)
cancelled_flights_df.to_sql(name='cancelled_flights', con=engine, if_exists='append', index=False)
print(f"Loaded {session.execute('select count(*) from cancelled_flights;').scalar():,} records to cancelled_flights table")

## Step 4: Confirm the data has been loaded into the database tables
### Table - cancellation_code
### Table - airlines
### Table - airports
### Table - cancelled_flights
### View - cancelled_flights_vw

In [22]:
pd.read_sql_query('select * from cancellation_code', con=engine).head()

,CANCELLATION_REASON_CODE,CANCELLATION_REASON
0,A,Airline/Carrier
1,B,Weather
2,C,National Air System
3,D,Security


In [23]:
pd.read_sql_query('select * from airlines', con=engine).head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [24]:
pd.read_sql_query('select * from airports', con=engine).head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,41.0,-75.0
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.0,-100.0
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.0,-107.0
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.0,-98.0
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,32.0,-84.0


In [25]:
pd.read_sql_query('select * from cancelled_flights', con=engine).head()

,ID,DATE,AIRLINE_CODE,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,CANCELLATION_REASON_CODE
0,0,2015-01-01 00:00:00,AS,ANC,SEA,A
1,1,2015-01-01 00:00:00,AA,PHX,DFW,B
2,2,2015-01-01 00:00:00,OO,MAF,IAH,B
3,3,2015-01-01 00:00:00,MQ,SGF,DFW,B
4,4,2015-01-01 00:00:00,OO,RDD,SFO,A


In [26]:
pd.read_sql_query('select * from cancelled_flights_vw', con=engine).head()

,ID,Date,Airline Code,Airline,Origin Airport Code,Origin Airport,Destination Airport Code,Destination Airport,Cancellation Code,Cancellation Reason
0,0,2015-01-01 00:00:00,AS,Alaska Airlines Inc.,ANC,"Ted Stevens Anchorage International Airport, A...",SEA,"Seattle-Tacoma International Airport, Seattle, WA",A,Airline/Carrier
1,1,2015-01-01 00:00:00,AA,American Airlines Inc.,PHX,"Phoenix Sky Harbor International Airport, Phoe...",DFW,"Dallas/Fort Worth International Airport, Dalla...",B,Weather
2,2,2015-01-01 00:00:00,OO,Skywest Airlines Inc.,MAF,"Midland International Airport, Midland, TX",IAH,"George Bush Intercontinental Airport, Houston, TX",B,Weather
3,3,2015-01-01 00:00:00,MQ,American Eagle Airlines Inc.,SGF,"Springfield-Branson National Airport, Springfi...",DFW,"Dallas/Fort Worth International Airport, Dalla...",B,Weather
4,4,2015-01-01 00:00:00,OO,Skywest Airlines Inc.,RDD,"Redding Municipal Airport, Redding, CA",SFO,"San Francisco International Airport, San Franc...",A,Airline/Carrier


# Additional Analysis and Visualizations Added 

In [29]:
gmaps.configure(api_key=gkey)

In [35]:
airports_df.dropna(inplace=True)
airports_df

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [36]:
locations = airports_df[["LATITUDE", "LONGITUDE"]]
locations

,LATITUDE,LONGITUDE
0,40.65236,-75.44040
1,32.41132,-99.68190
2,35.04022,-106.60919
3,45.44906,-98.42183
4,31.53552,-84.19447
...,...,...
317,56.48433,-132.36982
318,44.68840,-111.11764
319,36.28187,-94.30681
320,59.50336,-139.66023


In [43]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=4,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# View airport with most cancellations based on origin 
top_origin = cancelled_flights_df.groupby(cancelled_flights_df["ORIGIN_AIRPORT_CODE"]).count()
top_origin_sorted = top_origin.sort_values("CANCELLATION_REASON_CODE",ascending=False)
top_origin_sorted

In [ ]:
# View airport with most cancellations based on destination
top_destination = cancelled_flights_df.groupby(cancelled_flights_df["DESTINATION_AIRPORT_CODE"]).count()
top_destination_sorted = top_origin.sort_values("CANCELLATION_REASON_CODE",ascending=False)
top_destination_sorted

In [ ]:
# View airline with most cancellations 
top_airline = cancelled_flights_df.groupby(cancelled_flights_df["AIRLINE_CODE"]).count()
top_airline_sorted = top_airline.sort_values("CANCELLATION_REASON_CODE",ascending=False)
top_airline_sorted

In [ ]:
# View date with most cancellations 
top_date = cancelled_flights_df.groupby(cancelled_flights_df["DATE"]).count()
top_date_sorted = top_airline.sort_values("CANCELLATION_REASON_CODE",ascending=False)
top_date_sorted